In [0]:
# Databricks notebook source
# ================================================================
# 03_ingest_bronze - Parse dos XMLs e gravação em Delta Bronze
# ================================================================
from pyspark.sql import SparkSession
import xml.etree.ElementTree as ET
import os

In [0]:
spark = SparkSession.builder.getOrCreate()

dbfs_path = "/dbfs/mnt/amazonia/raw/xml/"
bronze_table = "amazonia_catalog.bronze.metadados"

files = [os.path.join(dbfs_path, f) for f in os.listdir(dbfs_path) if f.endswith(".xml")]

all_rows = []

In [0]:
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()

    # Ajustar a tag conforme estrutura real do XML
    for elem in root.findall(".//feature"):
        row = {
            "uuid": elem.findtext("uuid"),
            "state": elem.findtext("state"),
            "year": elem.findtext("year"),
            "area_km": elem.findtext("area_km"),
        }
        all_rows.append(row)

df = spark.createDataFrame(all_rows)

(
    df.write.format("delta")
    .mode("overwrite")   # use append em produção
    .saveAsTable(bronze_table)
)